In [1]:
import keras
from keras import backend as K
from keras.preprocessing import image
from keras.models import Model

import tensorflow as tf
from tensorflow.python.tools import freeze_graph, optimize_for_inference_lib

import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'



def get_model():
    vgg_model = keras.applications.VGG16(include_top=True, weights='imagenet')
    vgg_model.layers.pop()
    vgg_model.layers.pop()

    inp = vgg_model.input
    out = vgg_model.layers[-1].output

    model = Model(inp, out)
    return model

def get_features(model, cropped_image):
    x = image.img_to_array(cropped_image)
    x = np.expand_dims(x, axis=0)
    x = keras.applications.vgg16.preprocess_input(x)
    features = model.predict(x)
    return features

Using TensorFlow backend.


In [2]:
import numpy as np
np.random.seed(123)
import gzip
import _pickle as cPickle
import os
from collections import Counter

from sklearn.preprocessing import LabelEncoder, normalize
from sklearn.neighbors import KDTree

from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.utils import to_categorical


WORD2VECPATH    = "data/class_vectors.npy"
DATAPATH        = "data/zeroshot_data.pkl"
MODELPATH       = "model/"

def load_keras_model(model_path):
    with open(model_path +"model.json", 'r') as json_file:
        loaded_model_json = json_file.read()

    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(model_path+"model.h5")
    return loaded_model

def save_keras_model(model, model_path):
    """save Keras model and its weights"""
    if not os.path.exists(model_path):
        os.makedirs(model_path)

    model_json = model.to_json()
    with open(model_path + "model.json", "w") as json_file:
        json_file.write(model_json)

    # serialize weights to HDF5
    model.save_weights(model_path + "model.h5")
    print("-> zsl model is saved.")
    return



In [3]:
def load_data():
    """read data, create datasets"""
    # READ DATA
    with gzip.GzipFile(DATAPATH, 'rb') as infile:
        data = cPickle.load(infile)

    # ONE-HOT-ENCODE DATA
    label_encoder   = LabelEncoder()
    label_encoder.fit(train_classes)

    training_data = [instance for instance in data if instance[0] in train_classes]
    zero_shot_data = [instance for instance in data if instance[0] not in train_classes]
    # SHUFFLE TRAINING DATA
    np.random.shuffle(training_data)

    ### SPLIT DATA FOR TRAINING
    train_size  = 300
    train_data  = list()
    valid_data  = list()
    for class_label in train_classes:
        ct = 0
        for instance in training_data:
            if instance[0] == class_label:
                if ct < train_size:
                    train_data.append(instance)
                    ct+=1
                    continue
                valid_data.append(instance)

    # SHUFFLE TRAINING AND VALIDATION DATA
    np.random.shuffle(train_data)
    np.random.shuffle(valid_data)

    train_data = [(instance[1], to_categorical(label_encoder.transform([instance[0]]), num_classes=15))for instance in train_data]
    valid_data = [(instance[1], to_categorical(label_encoder.transform([instance[0]]), num_classes=15)) for instance in valid_data]

    # FORM X_TRAIN AND Y_TRAIN
    x_train, y_train    = zip(*train_data)
    x_train, y_train    = np.squeeze(np.asarray(x_train)), np.squeeze(np.asarray(y_train))
    # L2 NORMALIZE X_TRAIN
    x_train = normalize(x_train, norm='l2')

    # FORM X_VALID AND Y_VALID
    x_valid, y_valid = zip(*valid_data)
    x_valid, y_valid = np.squeeze(np.asarray(x_valid)), np.squeeze(np.asarray(y_valid))
    # L2 NORMALIZE X_VALID
    x_valid = normalize(x_valid, norm='l2')


    # FORM X_ZSL AND Y_ZSL
    y_zsl, x_zsl = zip(*zero_shot_data)
    x_zsl, y_zsl = np.squeeze(np.asarray(x_zsl)), np.squeeze(np.asarray(y_zsl))
    # L2 NORMALIZE X_ZSL
    x_zsl = normalize(x_zsl, norm='l2')

    print("-> data loading is completed.")
    return (x_train, x_valid, x_zsl), (y_train, y_valid, y_zsl)

In [4]:
def custom_kernel_init(shape):
    class_vectors       = np.load(WORD2VECPATH)
    training_vectors    = sorted([(label, vec) for (label, vec) in class_vectors if label in train_classes], key=lambda x: x[0])
    classnames, vectors = zip(*training_vectors)
    vectors             = np.asarray(vectors, dtype=np.float)
    vectors             = vectors.T
    return vectors

def  build_model():
    model = Sequential()
    model.add(Dense(1024, input_shape=(4096,), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.8))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(NUM_ATTR, activation='relu'))
    model.add(Dense(NUM_CLASS, activation='softmax', trainable=False, kernel_initializer=custom_kernel_init))

    print("-> model building is completed.")
    return model


def train_model(model, train_data, valid_data):
    x_train, y_train = train_data
    x_valid, y_valid = valid_data
    adam = Adam(lr=5e-5)
    model.compile(loss      = 'categorical_crossentropy',
                  optimizer = adam,
                  metrics   = ['categorical_accuracy', 'top_k_categorical_accuracy'])

    history = model.fit(x_train, y_train,
                        validation_data = (x_valid, y_valid),
                        verbose         = 2,
                        epochs          = EPOCH,
                        batch_size      = BATCH_SIZE,
                        shuffle         = True)

    print("model training is completed.")
    return history

def main():

    global train_classes
    with open('train_classes.txt', 'r') as infile:
        train_classes = [str.strip(line) for line in infile]

    global zsl_classes
    with open('zsl_classes.txt', 'r') as infile:
        zsl_classes = [str.strip(line) for line in infile]

    # ---------------------------------------------------------------------------------------------------------------- #
    # ---------------------------------------------------------------------------------------------------------------- #
    # SET HYPERPARAMETERS

    global NUM_CLASS, NUM_ATTR, EPOCH, BATCH_SIZE
    NUM_CLASS = 15
    NUM_ATTR = 300
    BATCH_SIZE = 128
    EPOCH = 65

    # ---------------------------------------------------------------------------------------------------------------- #
    # ---------------------------------------------------------------------------------------------------------------- #
    # TRAINING PHASE

    (x_train, x_valid, x_zsl), (y_train, y_valid, y_zsl) = load_data()
    model = build_model()
    train_model(model, (x_train, y_train), (x_valid, y_valid))
    print(model.summary())

    # ---------------------------------------------------------------------------------------------------------------- #
    # ---------------------------------------------------------------------------------------------------------------- #
    # CREATE AND SAVE ZSL MODEL

    inp         = model.input
    out         = model.layers[-2].output
    zsl_model   = Model(inp, out)
    print(zsl_model.summary())
    save_keras_model(zsl_model, model_path=MODELPATH)

    # ---------------------------------------------------------------------------------------------------------------- #
    # ---------------------------------------------------------------------------------------------------------------- #
    # EVALUATION OF ZERO-SHOT LEARNING PERFORMANCE
    #(x_train, x_valid, x_zsl), (y_train, y_valid, y_zsl) = load_data()
    #zsl_model = load_keras_model(model_path=MODELPATH)

    class_vectors       = sorted(np.load(WORD2VECPATH), key=lambda x: x[0])
    classnames, vectors = zip(*class_vectors)
    classnames          = list(classnames)
    vectors             = np.asarray(vectors, dtype=np.float)

    tree        = KDTree(vectors)
    pred_zsl    = zsl_model.predict(x_zsl)

    top5, top3, top1 = 0, 0, 0
    for i, pred in enumerate(pred_zsl):
        pred            = np.expand_dims(pred, axis=0)
        dist_5, index_5 = tree.query(pred, k=5)
        pred_labels     = [classnames[index] for index in index_5[0]]
        true_label      = y_zsl[i]
        if true_label in pred_labels:
            top5 += 1
        if true_label in pred_labels[:3]:
            top3 += 1
        if true_label in pred_labels[0]:
            top1 += 1

    print()
    print("ZERO SHOT LEARNING SCORE")
    print("-> Top-5 Accuracy: %.2f" % (top5 / float(len(x_zsl))))
    print("-> Top-3 Accuracy: %.2f" % (top3 / float(len(x_zsl))))
    print("-> Top-1 Accuracy: %.2f" % (top1 / float(len(x_zsl))))
    return

if __name__ == '__main__':
    main()

-> data loading is completed.
-> model building is completed.
Train on 4500 samples, validate on 1493 samples
Epoch 1/65
 - 1s - loss: 2.9265 - categorical_accuracy: 0.0904 - top_k_categorical_accuracy: 0.3900 - val_loss: 2.4661 - val_categorical_accuracy: 0.2465 - val_top_k_categorical_accuracy: 0.6859
Epoch 2/65
 - 0s - loss: 2.5952 - categorical_accuracy: 0.1502 - top_k_categorical_accuracy: 0.5278 - val_loss: 2.2519 - val_categorical_accuracy: 0.3262 - val_top_k_categorical_accuracy: 0.7810
Epoch 3/65
 - 0s - loss: 2.3967 - categorical_accuracy: 0.2098 - top_k_categorical_accuracy: 0.6200 - val_loss: 2.0624 - val_categorical_accuracy: 0.3724 - val_top_k_categorical_accuracy: 0.8359
Epoch 4/65
 - 0s - loss: 2.2412 - categorical_accuracy: 0.2480 - top_k_categorical_accuracy: 0.6873 - val_loss: 1.8967 - val_categorical_accuracy: 0.3938 - val_top_k_categorical_accuracy: 0.8654
Epoch 5/65
 - 0s - loss: 2.0793 - categorical_accuracy: 0.2860 - top_k_categorical_accuracy: 0.7564 - val_loss

Epoch 43/65
 - 0s - loss: 1.0068 - categorical_accuracy: 0.6227 - top_k_categorical_accuracy: 0.9631 - val_loss: 1.1531 - val_categorical_accuracy: 0.5693 - val_top_k_categorical_accuracy: 0.9431
Epoch 44/65
 - 0s - loss: 0.9767 - categorical_accuracy: 0.6342 - top_k_categorical_accuracy: 0.9669 - val_loss: 1.1492 - val_categorical_accuracy: 0.5740 - val_top_k_categorical_accuracy: 0.9457
Epoch 45/65
 - 0s - loss: 0.9888 - categorical_accuracy: 0.6291 - top_k_categorical_accuracy: 0.9636 - val_loss: 1.1487 - val_categorical_accuracy: 0.5834 - val_top_k_categorical_accuracy: 0.9444
Epoch 46/65
 - 0s - loss: 0.9603 - categorical_accuracy: 0.6404 - top_k_categorical_accuracy: 0.9664 - val_loss: 1.1461 - val_categorical_accuracy: 0.5807 - val_top_k_categorical_accuracy: 0.9451
Epoch 47/65
 - 0s - loss: 0.9769 - categorical_accuracy: 0.6407 - top_k_categorical_accuracy: 0.9658 - val_loss: 1.1459 - val_categorical_accuracy: 0.5800 - val_top_k_categorical_accuracy: 0.9417
Epoch 48/65
 - 0s - 

In [2]:
#
# detect_object.py
#
# Created by Samet Cetin.
# Contact: cetin.samet@outlook.com
#

import sys
import numpy as np
from PIL import Image

from sklearn.neighbors import KDTree
from sklearn.preprocessing import normalize

from feature_extractor import get_model, get_features
from train import load_keras_model


WORD2VECPATH    = "data/class_vectors.npy"
MODELPATH       = "model/"

def main(argv):

    if len(argv) != 1:
        print("Usage: python3 detect_object.py input-image-path")
        exit()

    # READ IMAGE
    IMAGEPATH = 'test6.jpg'
    img         = Image.open(IMAGEPATH).resize((224, 224))

    # LOAD PRETRAINED VGG16 MODEL FOR FEATURE EXTRACTION
    vgg_model   = get_model()
    # EXTRACT IMAGE FEATURE
    img_feature = get_features(vgg_model, img)
    
    print(img_feature.shape)
    # L2 NORMALIZE FEATURE
    img_feature = normalize(img_feature, norm='l2')
    
    print(img_feature)
    
    # LOAD ZERO-SHOT MODEL
    model       = load_keras_model(model_path=MODELPATH)
    # MAKE PREDICTION
    pred        = model.predict(img_feature)

    # LOAD CLASS WORD2VECS
    class_vectors       = sorted(np.load(WORD2VECPATH), key=lambda x: x[0])
    classnames, vectors = zip(*class_vectors)
    classnames          = list(classnames)
    vectors             = np.asarray(vectors, dtype=np.float)

    # PLACE WORD2VECS IN KDTREE
    tree                = KDTree(vectors)
    # FIND CLOSEST WORD2VEC and GET PREDICTION RESULT
    dist, index         = tree.query(pred, k=5)
    pred_labels         = [classnames[idx] for idx in index[0]]

    # PRINT RESULT
    print()
    print("--- Top-5 Prediction ---")
    for i, classname in enumerate(pred_labels):
        print("%d- %s" %(i+1, classname))
    print()
    return

if __name__ == '__main__':
    main(sys.argv[1:])

Usage: python3 detect_object.py input-image-path
(1, 4096)
[[0.00744639 0.05892316 0.         ... 0.         0.         0.        ]]

--- Top-5 Prediction ---
1- computer
2- sandwich
3- food
4- woman
5- child

